In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.getOrCreate()

In [4]:
retail = spark.read.format("csv")\
.option('header', True)\
.option("inferschema", True)\
.load("online_retail_data.csv")

In [5]:
retail.show(5)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|01/12/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01/12/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01/12/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|01/12/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01/12/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows



1.feladat: Átlagosan hány különböző terméket vesz egy vásárló egy vásárlás során?

In [9]:
retail\
.groupBy("InvoiceNo")\
.agg(countDistinct("StockCode").alias("db"))\
.select(avg("db"))\
.show()

+------------------+
|           avg(db)|
+------------------+
|20.514131274131273|
+------------------+



2.feladat: Melyik tranzakció során vásárolták a legtöbb különböző terméket és hányat?

In [13]:
retail\
.groupBy("InvoiceNo")\
.agg(countDistinct("StockCode").alias("db"))\
.orderBy(desc("db"))\
.limit(1)\
.show()

+---------+----+
|InvoiceNo|  db|
+---------+----+
|   573585|1110|
+---------+----+



3.feladat: Melyik termék termelte a legnagyobb bevételt?

In [16]:
from pyspark.sql.functions import col

retail.selectExpr('*', '(Quantity * UnitPrice) as bevetel')\
.groupBy('StockCode')\
.sum('bevetel')\
.orderBy(col('sum(bevetel)').desc() )\
.limit(1)\
.show()

+---------+------------------+
|StockCode|      sum(bevetel)|
+---------+------------------+
|      DOT|206245.48000000004|
+---------+------------------+

